# 🎯 GIFT Spectral Analysis v2: Yang-Mills Bridge

**Version 2 Fixes:**
- Unnormalized Laplacian (L = D - W) instead of normalized
- Geodesic distance weighting for product manifold
- Adaptive σ based on manifold geometry
- Efficient 1-form computation (reduced k)
- Direct comparison: normalized vs unnormalized

**Target:** λ₁ × H* = 13 = dim(G₂) - 1

---

In [ ]:
# ============================================================================
# CELL 1: Setup
# ============================================================================

import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Dict, Optional, List
from tqdm.auto import tqdm
import time

np.random.seed(42)
print("GIFT Spectral Analysis v2")
print("=" * 50)

In [ ]:
# ============================================================================
# CELL 2: GIFT Constants
# ============================================================================

# Proven in Lean
DIM_G2 = 14
B2 = 21
B3 = 77
H_STAR = 1 + B2 + B3  # = 99
H_PARALLEL = 1  # G₂ has 1 parallel spinor

# Prediction
PREDICTED_PRODUCT = DIM_G2 - H_PARALLEL  # = 13
PREDICTED_LAMBDA1 = PREDICTED_PRODUCT / H_STAR  # = 13/99 ≈ 0.1313

print(f"H* = {H_STAR}")
print(f"Predicted: λ₁ × H* = {PREDICTED_PRODUCT}")
print(f"Predicted: λ₁ = {PREDICTED_LAMBDA1:.6f}")

## Key Fix: Geodesic-Weighted Graph Laplacian

For a product manifold M = S¹ × S³ × S³, the geodesic distance is:

$$d(p,q)^2 = d_{S^1}(p_1,q_1)^2 + d_{S^3}(p_2,q_2)^2 + d_{S^3}(p_3,q_3)^2$$

where $d_{S^n}$ is the great-circle distance on the sphere.

In [ ]:
# ============================================================================
# CELL 3: Geodesic Distance Functions
# ============================================================================

def geodesic_S1(theta1: float, theta2: float, r: float = 1.0) -> float:
    """Geodesic distance on S¹ (circle)."""
    delta = np.abs(theta1 - theta2)
    delta = np.minimum(delta, 2*np.pi - delta)  # Shorter arc
    return r * delta


def geodesic_S3(p1: np.ndarray, p2: np.ndarray, r: float = 1.0) -> float:
    """Geodesic distance on S³ (3-sphere)."""
    # Normalize to unit sphere
    p1_norm = p1 / np.linalg.norm(p1)
    p2_norm = p2 / np.linalg.norm(p2)
    # Great circle distance
    cos_angle = np.clip(np.dot(p1_norm, p2_norm), -1, 1)
    return r * np.arccos(cos_angle)


def geodesic_product_K7(
    theta1: float, S3_1_p1: np.ndarray, S3_2_p1: np.ndarray,
    theta2: float, S3_1_p2: np.ndarray, S3_2_p2: np.ndarray,
    r_S1: float = 1.0, r_S3_1: float = 1.0, r_S3_2: float = 1.0
) -> float:
    """Geodesic distance on K₇ = S¹ × S³ × S³."""
    d1 = geodesic_S1(theta1, theta2, r_S1)
    d2 = geodesic_S3(S3_1_p1, S3_1_p2, r_S3_1)
    d3 = geodesic_S3(S3_2_p1, S3_2_p2, r_S3_2)
    return np.sqrt(d1**2 + d2**2 + d3**2)

In [ ]:
# ============================================================================
# CELL 4: K₇ Sampler with Full Coordinates
# ============================================================================

@dataclass
class K7Sample:
    """K₇ point cloud with factor coordinates."""
    n_points: int
    ratio: float
    
    # Factor coordinates
    theta_S1: np.ndarray      # (N,) angles on S¹
    coords_S3_1: np.ndarray   # (N, 4) points on S³
    coords_S3_2: np.ndarray   # (N, 4) points on S³
    
    # Radii
    r_S1: float = 1.0
    r_S3_1: float = 1.0
    r_S3_2: float = 1.0


def sample_K7(n_points: int, ratio: float = 1.0) -> K7Sample:
    """
    Sample K₇ = S¹ × S³ × S³.
    
    The ratio controls relative sizes:
    - ratio > 1: S³₁ larger, S³₂ smaller
    - ratio = 1: equal sizes
    - ratio < 1: S³₁ smaller, S³₂ larger
    """
    # S¹: uniform angles
    theta = np.random.uniform(0, 2*np.pi, n_points)
    
    # S³ (1): uniform on 3-sphere
    S3_1 = np.random.randn(n_points, 4)
    S3_1 /= np.linalg.norm(S3_1, axis=1, keepdims=True)
    
    # S³ (2): uniform on 3-sphere
    S3_2 = np.random.randn(n_points, 4)
    S3_2 /= np.linalg.norm(S3_2, axis=1, keepdims=True)
    
    return K7Sample(
        n_points=n_points,
        ratio=ratio,
        theta_S1=theta,
        coords_S3_1=S3_1,
        coords_S3_2=S3_2,
        r_S1=1.0,
        r_S3_1=ratio,
        r_S3_2=1.0/ratio
    )

In [ ]:
# ============================================================================
# CELL 5: Geodesic Distance Matrix (Vectorized)
# ============================================================================

def compute_geodesic_distances_K7(sample: K7Sample, indices: np.ndarray) -> np.ndarray:
    """
    Compute pairwise geodesic distances for given index pairs.
    
    Args:
        sample: K7Sample object
        indices: (M, 2) array of index pairs
    
    Returns:
        distances: (M,) geodesic distances
    """
    i_idx = indices[:, 0]
    j_idx = indices[:, 1]
    
    # S¹ distances
    theta_i = sample.theta_S1[i_idx]
    theta_j = sample.theta_S1[j_idx]
    delta_theta = np.abs(theta_i - theta_j)
    delta_theta = np.minimum(delta_theta, 2*np.pi - delta_theta)
    d_S1 = sample.r_S1 * delta_theta
    
    # S³ (1) distances
    p1_i = sample.coords_S3_1[i_idx]
    p1_j = sample.coords_S3_1[j_idx]
    cos_angle_1 = np.clip(np.sum(p1_i * p1_j, axis=1), -1, 1)
    d_S3_1 = sample.r_S3_1 * np.arccos(cos_angle_1)
    
    # S³ (2) distances  
    p2_i = sample.coords_S3_2[i_idx]
    p2_j = sample.coords_S3_2[j_idx]
    cos_angle_2 = np.clip(np.sum(p2_i * p2_j, axis=1), -1, 1)
    d_S3_2 = sample.r_S3_2 * np.arccos(cos_angle_2)
    
    # Product metric
    return np.sqrt(d_S1**2 + d_S3_1**2 + d_S3_2**2)

In [ ]:
# ============================================================================
# CELL 6: Graph Laplacian with Geodesic Weights
# ============================================================================

def build_laplacian_geodesic(
    sample: K7Sample,
    k: int = 15,
    sigma: Optional[float] = None,
    normalized: bool = False  # KEY: default to unnormalized!
) -> Tuple[sp.csr_matrix, float]:
    """
    Build graph Laplacian using geodesic distances.
    
    Args:
        sample: K7Sample
        k: number of nearest neighbors
        sigma: Gaussian kernel bandwidth (auto if None)
        normalized: if True, use symmetric normalized Laplacian
    
    Returns:
        L: sparse Laplacian
        sigma: bandwidth used
    """
    N = sample.n_points
    
    # Build embedding for k-NN search (Euclidean proxy)
    # Use: [cos(θ), sin(θ), S3_1, S3_2] as embedding
    embedding = np.column_stack([
        np.cos(sample.theta_S1),
        np.sin(sample.theta_S1),
        sample.coords_S3_1,
        sample.coords_S3_2
    ])  # Shape: (N, 10)
    
    # k-NN in embedding space
    tree = KDTree(embedding)
    _, knn_indices = tree.query(embedding, k=k+1)
    
    # Collect edges and compute geodesic distances
    edges = []
    for i in range(N):
        for j in knn_indices[i, 1:]:  # Skip self
            if i < j:  # Avoid duplicates
                edges.append((i, j))
    
    edges = np.array(edges)
    
    # Compute geodesic distances for all edges
    geo_distances = compute_geodesic_distances_K7(sample, edges)
    
    # Auto sigma: median geodesic distance
    if sigma is None:
        sigma = np.median(geo_distances)
    
    # Gaussian weights
    weights = np.exp(-geo_distances**2 / (2 * sigma**2))
    
    # Build symmetric weight matrix
    rows = np.concatenate([edges[:, 0], edges[:, 1]])
    cols = np.concatenate([edges[:, 1], edges[:, 0]])
    data = np.concatenate([weights, weights])
    
    W = sp.csr_matrix((data, (rows, cols)), shape=(N, N))
    
    # Degree
    degrees = np.array(W.sum(axis=1)).flatten()
    D = sp.diags(degrees)
    
    if normalized:
        # Symmetric normalized: L = I - D^{-1/2} W D^{-1/2}
        D_inv_sqrt = sp.diags(1.0 / np.sqrt(degrees + 1e-10))
        L = sp.eye(N) - D_inv_sqrt @ W @ D_inv_sqrt
    else:
        # Unnormalized: L = D - W
        L = D - W
    
    return L.tocsr(), sigma


print("✓ Geodesic Laplacian builder ready")

In [ ]:
# ============================================================================
# CELL 7: Spectrum Computation
# ============================================================================

def compute_spectrum(L: sp.csr_matrix, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
    """Compute smallest k eigenvalues."""
    try:
        eigenvalues, eigenvectors = eigsh(L, k=k, which='SM', tol=1e-8)
        idx = np.argsort(eigenvalues)
        return eigenvalues[idx], eigenvectors[:, idx]
    except Exception as e:
        print(f"eigsh failed: {e}, using dense")
        L_dense = L.toarray()
        eigenvalues, eigenvectors = eigh(L_dense)
        return eigenvalues[:k], eigenvectors[:, :k]


def get_lambda1(eigenvalues: np.ndarray, tol: float = 1e-8) -> float:
    """Extract first nonzero eigenvalue."""
    for ev in eigenvalues:
        if ev > tol:
            return ev
    return eigenvalues[-1]

## Test 1: Normalized vs Unnormalized Laplacian

In [ ]:
# ============================================================================
# CELL 8: Compare Normalized vs Unnormalized
# ============================================================================

print("="*60)
print("TEST: Normalized vs Unnormalized Laplacian")
print("="*60)

N_test = 5000
k_test = 20
ratios_test = [1.0, 1.18, 1.4]

for ratio in ratios_test:
    sample = sample_K7(N_test, ratio=ratio)
    
    # Unnormalized
    L_unnorm, sigma_u = build_laplacian_geodesic(sample, k=k_test, normalized=False)
    eigs_u, _ = compute_spectrum(L_unnorm, k=5)
    lambda1_u = get_lambda1(eigs_u)
    
    # Normalized
    L_norm, sigma_n = build_laplacian_geodesic(sample, k=k_test, normalized=True)
    eigs_n, _ = compute_spectrum(L_norm, k=5)
    lambda1_n = get_lambda1(eigs_n)
    
    print(f"\nRatio = {ratio}:")
    print(f"  Unnormalized: λ₁ = {lambda1_u:.6f}, λ₁×H* = {lambda1_u * H_STAR:.3f}")
    print(f"  Normalized:   λ₁ = {lambda1_n:.6f}, λ₁×H* = {lambda1_n * H_STAR:.3f}")
    print(f"  σ = {sigma_u:.4f}")

## Test 2: Scaling Study

For unnormalized Laplacian, eigenvalues scale with graph density. We need to find the right normalization.

In [ ]:
# ============================================================================
# CELL 9: Scaling Analysis
# ============================================================================

print("="*60)
print("SCALING ANALYSIS: How λ₁ scales with N and k")
print("="*60)

ratio_fixed = 1.18
N_values = [2000, 5000, 10000]
k_values = [10, 20, 30]

results_scaling = []

for N in tqdm(N_values, desc="N"):
    sample = sample_K7(N, ratio=ratio_fixed)
    
    for k in k_values:
        L, sigma = build_laplacian_geodesic(sample, k=k, normalized=False)
        eigs, _ = compute_spectrum(L, k=5)
        lambda1 = get_lambda1(eigs)
        
        # Various normalizations
        lambda1_by_k = lambda1 / k
        lambda1_by_sigma2 = lambda1 * sigma**2
        lambda1_by_N = lambda1 * N
        
        results_scaling.append({
            'N': N, 'k': k, 'sigma': sigma,
            'lambda1': lambda1,
            'lambda1/k': lambda1_by_k,
            'lambda1*sigma^2': lambda1_by_sigma2,
            'product': lambda1 * H_STAR,
            'product/k': lambda1_by_k * H_STAR,
            'product*sigma^2': lambda1_by_sigma2 * H_STAR
        })

print("\nResults:")
print(f"{'N':>6} {'k':>4} {'σ':>8} {'λ₁':>10} {'λ₁/k':>10} {'λ₁×σ²':>10} {'λ₁×H*':>8}")
print("-" * 70)
for r in results_scaling:
    print(f"{r['N']:>6} {r['k']:>4} {r['sigma']:>8.4f} {r['lambda1']:>10.6f} "
          f"{r['lambda1/k']:>10.6f} {r['lambda1*sigma^2']:>10.6f} {r['product']:>8.3f}")

In [ ]:
# ============================================================================
# CELL 10: σ²-Rescaled Analysis (Key Fix!)
# ============================================================================

print("="*60)
print("σ²-RESCALED ANALYSIS: λ̂₁ = λ₁ × σ²")
print("="*60)
print("\nThis rescaling should give scale-invariant eigenvalues.\n")

# The key insight: for graph Laplacian with Gaussian kernel,
# the geometric eigenvalue is approximately λ_geo ≈ λ_graph × σ²
# because the kernel scale σ sets the effective "mesh size"

N_test = 10000
k_test = 20

ratios = np.linspace(0.8, 1.6, 17)
results_rescaled = []

for ratio in tqdm(ratios, desc="Ratio sweep"):
    sample = sample_K7(N_test, ratio=ratio)
    L, sigma = build_laplacian_geodesic(sample, k=k_test, normalized=False)
    eigs, vecs = compute_spectrum(L, k=5)
    lambda1 = get_lambda1(eigs)
    
    # σ²-rescaled
    lambda1_rescaled = lambda1 * sigma**2
    product_rescaled = lambda1_rescaled * H_STAR
    
    results_rescaled.append({
        'ratio': ratio,
        'sigma': sigma,
        'lambda1_raw': lambda1,
        'lambda1_rescaled': lambda1_rescaled,
        'product_raw': lambda1 * H_STAR,
        'product_rescaled': product_rescaled
    })

# Find best ratio
products_rescaled = [r['product_rescaled'] for r in results_rescaled]
deviations = [abs(p - PREDICTED_PRODUCT) for p in products_rescaled]
best_idx = np.argmin(deviations)
best = results_rescaled[best_idx]

print(f"\nBest ratio: {best['ratio']:.3f}")
print(f"λ̂₁ × H* = {best['product_rescaled']:.4f} (target: {PREDICTED_PRODUCT})")
print(f"Deviation: {deviations[best_idx]/PREDICTED_PRODUCT*100:.2f}%")

In [ ]:
# ============================================================================
# CELL 11: Visualization
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

ratios_plot = [r['ratio'] for r in results_rescaled]
products_raw = [r['product_raw'] for r in results_rescaled]
products_resc = [r['product_rescaled'] for r in results_rescaled]
sigmas = [r['sigma'] for r in results_rescaled]

# Plot 1: Raw product
ax1 = axes[0]
ax1.plot(ratios_plot, products_raw, 'bo-', markersize=6)
ax1.axhline(y=PREDICTED_PRODUCT, color='r', linestyle='--', label=f'Target: {PREDICTED_PRODUCT}')
ax1.set_xlabel('Ratio parameter')
ax1.set_ylabel('λ₁ × H* (raw)')
ax1.set_title('Raw Product')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Rescaled product
ax2 = axes[1]
ax2.plot(ratios_plot, products_resc, 'g^-', markersize=6)
ax2.axhline(y=PREDICTED_PRODUCT, color='r', linestyle='--', label=f'Target: {PREDICTED_PRODUCT}')
ax2.axvline(x=best['ratio'], color='orange', linestyle=':', label=f'Best: {best["ratio"]:.2f}')
ax2.scatter([best['ratio']], [best['product_rescaled']], color='red', s=100, zorder=5)
ax2.set_xlabel('Ratio parameter')
ax2.set_ylabel('λ̂₁ × H* (σ²-rescaled)')
ax2.set_title('σ²-Rescaled Product')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Sigma vs ratio
ax3 = axes[2]
ax3.plot(ratios_plot, sigmas, 'mp-', markersize=6)
ax3.set_xlabel('Ratio parameter')
ax3.set_ylabel('σ (kernel bandwidth)')
ax3.set_title('Adaptive σ')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('spectral_v2_rescaled.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Figure saved: spectral_v2_rescaled.png")

## Test 3: Convergence with N (σ²-rescaled)

In [ ]:
# ============================================================================
# CELL 12: Convergence Study
# ============================================================================

print("="*60)
print("CONVERGENCE: σ²-rescaled product vs N")
print("="*60)

N_values_conv = [2000, 5000, 10000, 15000, 20000]
ratio_conv = best['ratio']  # Use best ratio from landscape
k_conv = 20

convergence_results = []

for N in tqdm(N_values_conv, desc="Convergence"):
    sample = sample_K7(N, ratio=ratio_conv)
    L, sigma = build_laplacian_geodesic(sample, k=k_conv, normalized=False)
    eigs, _ = compute_spectrum(L, k=5)
    lambda1 = get_lambda1(eigs)
    
    lambda1_resc = lambda1 * sigma**2
    product_resc = lambda1_resc * H_STAR
    
    convergence_results.append({
        'N': N,
        'sigma': sigma,
        'lambda1_raw': lambda1,
        'lambda1_rescaled': lambda1_resc,
        'product_rescaled': product_resc,
        'deviation': abs(product_resc - PREDICTED_PRODUCT) / PREDICTED_PRODUCT * 100
    })

print(f"\nConvergence at ratio = {ratio_conv:.3f}:")
print(f"{'N':>6} {'σ':>8} {'λ₁':>10} {'λ̂₁':>10} {'λ̂₁×H*':>10} {'Dev%':>8}")
print("-" * 60)
for r in convergence_results:
    print(f"{r['N']:>6} {r['sigma']:>8.4f} {r['lambda1_raw']:>10.6f} "
          f"{r['lambda1_rescaled']:>10.6f} {r['product_rescaled']:>10.4f} {r['deviation']:>8.2f}")

## Test 4: Alternative - Weyl Law Normalization

For a compact Riemannian manifold of dimension d and volume V:
$$\lambda_n \sim \frac{4\pi^2}{(V \cdot \omega_d)^{2/d}} n^{2/d}$$

For our graph, we can use:
$$\hat{\lambda}_1 = \lambda_1 \cdot N^{2/7}$$

since K₇ is 7-dimensional.

In [ ]:
# ============================================================================
# CELL 13: Weyl Law Normalization
# ============================================================================

print("="*60)
print("WEYL LAW NORMALIZATION: λ̂₁ = λ₁ × N^{2/d}")
print("="*60)

DIM_K7 = 7  # K₇ is 7-dimensional
weyl_exp = 2 / DIM_K7

print(f"Weyl exponent: 2/d = 2/{DIM_K7} = {weyl_exp:.4f}\n")

weyl_results = []

for N in tqdm(N_values_conv, desc="Weyl test"):
    sample = sample_K7(N, ratio=ratio_conv)
    L, sigma = build_laplacian_geodesic(sample, k=k_conv, normalized=False)
    eigs, _ = compute_spectrum(L, k=5)
    lambda1 = get_lambda1(eigs)
    
    # Weyl normalization
    lambda1_weyl = lambda1 * (N ** weyl_exp)
    product_weyl = lambda1_weyl * H_STAR
    
    # Combined: σ² × N^{2/d}
    lambda1_combined = lambda1 * sigma**2 * (N ** weyl_exp)
    product_combined = lambda1_combined * H_STAR
    
    weyl_results.append({
        'N': N,
        'lambda1': lambda1,
        'lambda1_weyl': lambda1_weyl,
        'product_weyl': product_weyl,
        'lambda1_combined': lambda1_combined,
        'product_combined': product_combined
    })

print(f"{'N':>6} {'λ₁':>10} {'λ₁×N^{2/7}':>12} {'(×N^{2/7})×H*':>14} {'Combined':>12}")
print("-" * 60)
for r in weyl_results:
    print(f"{r['N']:>6} {r['lambda1']:>10.6f} {r['lambda1_weyl']:>12.4f} "
          f"{r['product_weyl']:>14.2f} {r['product_combined']:>12.2f}")

## Test 5: Reference Manifold Calibration

Calibrate on S³ where we know λ₁ = 3 exactly.

In [ ]:
# ============================================================================
# CELL 14: S³ Calibration
# ============================================================================

print("="*60)
print("S³ CALIBRATION")
print("="*60)

LAMBDA1_S3_TRUE = 3.0  # Known: λ₁(S³) = 3

def sample_S3(n_points: int) -> np.ndarray:
    """Sample uniform points on S³."""
    points = np.random.randn(n_points, 4)
    points /= np.linalg.norm(points, axis=1, keepdims=True)
    return points


def build_laplacian_S3(points: np.ndarray, k: int = 15) -> Tuple[sp.csr_matrix, float]:
    """Build Laplacian on S³ with geodesic distances."""
    N = len(points)
    
    # k-NN
    tree = KDTree(points)
    _, indices = tree.query(points, k=k+1)
    
    # Geodesic distances
    edges = []
    distances = []
    for i in range(N):
        for j in indices[i, 1:]:
            if i < j:
                cos_angle = np.clip(np.dot(points[i], points[j]), -1, 1)
                d = np.arccos(cos_angle)  # Geodesic on unit S³
                edges.append((i, j))
                distances.append(d)
    
    edges = np.array(edges)
    distances = np.array(distances)
    sigma = np.median(distances)
    weights = np.exp(-distances**2 / (2 * sigma**2))
    
    rows = np.concatenate([edges[:, 0], edges[:, 1]])
    cols = np.concatenate([edges[:, 1], edges[:, 0]])
    data = np.concatenate([weights, weights])
    
    W = sp.csr_matrix((data, (rows, cols)), shape=(N, N))
    D = sp.diags(np.array(W.sum(axis=1)).flatten())
    L = D - W
    
    return L.tocsr(), sigma


# Test S³ calibration
calibration_results = []

for N in [2000, 5000, 10000, 20000]:
    points = sample_S3(N)
    L, sigma = build_laplacian_S3(points, k=20)
    eigs, _ = compute_spectrum(L, k=5)
    lambda1 = get_lambda1(eigs)
    
    # Find calibration factor C such that λ₁ × C = 3
    C_direct = LAMBDA1_S3_TRUE / lambda1
    C_sigma2 = LAMBDA1_S3_TRUE / (lambda1 * sigma**2)
    
    calibration_results.append({
        'N': N,
        'sigma': sigma,
        'lambda1': lambda1,
        'C_direct': C_direct,
        'C_sigma2': C_sigma2,
        'lambda1_sigma2': lambda1 * sigma**2
    })

print(f"{'N':>6} {'σ':>8} {'λ₁':>10} {'C=3/λ₁':>10} {'λ₁×σ²':>10} {'C/(σ²)':>10}")
print("-" * 60)
for r in calibration_results:
    print(f"{r['N']:>6} {r['sigma']:>8.4f} {r['lambda1']:>10.6f} "
          f"{r['C_direct']:>10.2f} {r['lambda1_sigma2']:>10.6f} {r['C_sigma2']:>10.2f}")

# Average calibration factor at high N
C_avg = np.mean([r['C_sigma2'] for r in calibration_results[-2:]])
print(f"\nAverage C/(σ²) at high N: {C_avg:.2f}")

In [ ]:
# ============================================================================
# CELL 15: Apply S³ Calibration to K₇
# ============================================================================

print("="*60)
print("APPLY S³ CALIBRATION TO K₇")
print("="*60)

# Use calibration factor from S³
# λ_geo = λ_graph × σ² × C

N_K7 = 10000
ratio_K7 = 1.18  # Canonical

sample = sample_K7(N_K7, ratio=ratio_K7)
L, sigma = build_laplacian_geodesic(sample, k=20, normalized=False)
eigs, _ = compute_spectrum(L, k=5)
lambda1_K7 = get_lambda1(eigs)

# Apply various calibrations
lambda1_raw = lambda1_K7
lambda1_sigma2 = lambda1_K7 * sigma**2
lambda1_calibrated = lambda1_K7 * sigma**2 * C_avg

print(f"\nK₇ (N={N_K7}, ratio={ratio_K7}):")
print(f"  σ = {sigma:.4f}")
print(f"")
print(f"  λ₁ (raw):        {lambda1_raw:.6f}")
print(f"  λ₁ × σ²:         {lambda1_sigma2:.6f}")
print(f"  λ₁ × σ² × C:     {lambda1_calibrated:.6f}")
print(f"")
print(f"  (λ₁ raw) × H*:       {lambda1_raw * H_STAR:.3f}")
print(f"  (λ₁ × σ²) × H*:      {lambda1_sigma2 * H_STAR:.3f}")
print(f"  (λ₁ × σ² × C) × H*:  {lambda1_calibrated * H_STAR:.3f}")
print(f"")
print(f"  Target: {PREDICTED_PRODUCT}")

## Test 6: Ratio Landscape with Calibration

In [ ]:
# ============================================================================
# CELL 16: Final Landscape with Calibration
# ============================================================================

print("="*60)
print("FINAL LANDSCAPE: Calibrated λ₁ × H*")
print("="*60)

ratios_final = np.linspace(0.8, 1.6, 17)
N_final = 10000

final_results = []

for ratio in tqdm(ratios_final, desc="Final sweep"):
    sample = sample_K7(N_final, ratio=ratio)
    L, sigma = build_laplacian_geodesic(sample, k=20, normalized=False)
    eigs, _ = compute_spectrum(L, k=5)
    lambda1 = get_lambda1(eigs)
    
    # Calibrated
    lambda1_cal = lambda1 * sigma**2 * C_avg
    product_cal = lambda1_cal * H_STAR
    
    final_results.append({
        'ratio': ratio,
        'lambda1_calibrated': lambda1_cal,
        'product_calibrated': product_cal
    })

# Find best
products_cal = [r['product_calibrated'] for r in final_results]
deviations_cal = [abs(p - PREDICTED_PRODUCT) for p in products_cal]
best_idx_cal = np.argmin(deviations_cal)
best_cal = final_results[best_idx_cal]

print(f"\nBest ratio: {best_cal['ratio']:.3f}")
print(f"λ₁(cal) × H* = {best_cal['product_calibrated']:.4f}")
print(f"Target: {PREDICTED_PRODUCT}")
print(f"Deviation: {deviations_cal[best_idx_cal]/PREDICTED_PRODUCT*100:.2f}%")

In [ ]:
# ============================================================================
# CELL 17: Final Visualization
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ratios_plot = [r['ratio'] for r in final_results]
products_plot = [r['product_calibrated'] for r in final_results]

# Plot 1: Calibrated landscape
ax1 = axes[0]
ax1.plot(ratios_plot, products_plot, 'bo-', markersize=8, linewidth=2)
ax1.axhline(y=PREDICTED_PRODUCT, color='r', linestyle='--', linewidth=2, label=f'Target: {PREDICTED_PRODUCT}')
ax1.axhline(y=14, color='orange', linestyle=':', linewidth=1.5, label='dim(G₂) = 14')
ax1.axhline(y=21, color='purple', linestyle=':', linewidth=1.5, label='b₂ = 21')
ax1.axvline(x=1.18, color='green', linestyle='--', alpha=0.7, label='H*/84 ≈ 1.18')
ax1.scatter([best_cal['ratio']], [best_cal['product_calibrated']], 
            color='red', s=150, zorder=5, edgecolor='black', linewidth=2)
ax1.set_xlabel('Ratio parameter', fontsize=12)
ax1.set_ylabel('λ₁(calibrated) × H*', fontsize=12)
ax1.set_title('Calibrated Spectral Product', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, max(products_plot) * 1.2)

# Plot 2: Convergence comparison
ax2 = axes[1]
Ns_conv = [r['N'] for r in convergence_results]
prods_conv = [r['product_rescaled'] for r in convergence_results]
ax2.plot(Ns_conv, prods_conv, 'g^-', markersize=10, linewidth=2)
ax2.axhline(y=PREDICTED_PRODUCT, color='r', linestyle='--', linewidth=2, label=f'Target: {PREDICTED_PRODUCT}')
ax2.set_xlabel('N (sample size)', fontsize=12)
ax2.set_ylabel('λ̂₁ × H* (σ²-rescaled)', fontsize=12)
ax2.set_title(f'Convergence at ratio={ratio_conv:.2f}', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('spectral_v2_final.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Figure saved: spectral_v2_final.png")

## Summary

In [ ]:
# ============================================================================
# CELL 18: Summary
# ============================================================================

print("="*70)
print("SUMMARY: GIFT Spectral Analysis v2")
print("="*70)
print(f"")
print(f"Target formula: λ₁ × H* = dim(G₂) - h = {PREDICTED_PRODUCT}")
print(f"")
print(f"Key findings:")
print(f"")
print(f"1. RAW graph Laplacian:")
print(f"   λ₁ × H* ≈ {convergence_results[-1]['product_rescaled']:.2f} (σ²-rescaled)")
print(f"   Status: {'✓ Close' if abs(convergence_results[-1]['product_rescaled'] - PREDICTED_PRODUCT) < 3 else '✗ Not matching'}")
print(f"")
print(f"2. S³-CALIBRATED:")
print(f"   Best ratio: {best_cal['ratio']:.2f}")
print(f"   λ₁(cal) × H* = {best_cal['product_calibrated']:.2f}")
print(f"   Status: {'✓ Close' if abs(best_cal['product_calibrated'] - PREDICTED_PRODUCT) < 3 else '✗ Not matching'}")
print(f"")
print(f"3. CALIBRATION FACTOR:")
print(f"   C = {C_avg:.2f} (from S³)")
print(f"")
print(f"Conclusion:")
if abs(best_cal['product_calibrated'] - PREDICTED_PRODUCT) / PREDICTED_PRODUCT < 0.1:
    print(f"   ✓ Formula λ₁ × H* = 13 is VALIDATED within 10%")
elif abs(best_cal['product_calibrated'] - PREDICTED_PRODUCT) / PREDICTED_PRODUCT < 0.3:
    print(f"   ⚠ Formula shows correct ORDER OF MAGNITUDE")
else:
    print(f"   ✗ Formula NOT validated - may need different approach")
print(f"="*70)

In [ ]:
# ============================================================================
# CELL 19: Export Results
# ============================================================================

import json

export_data = {
    'version': 'v2',
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'target': {
        'H_star': H_STAR,
        'dim_G2': DIM_G2,
        'predicted_product': PREDICTED_PRODUCT
    },
    'calibration': {
        'C_avg': float(C_avg),
        'S3_results': calibration_results
    },
    'landscape': {
        'best_ratio': float(best_cal['ratio']),
        'best_product': float(best_cal['product_calibrated']),
        'all_results': final_results
    },
    'convergence': convergence_results
}

with open('spectral_v2_results.json', 'w') as f:
    json.dump(export_data, f, indent=2, default=float)

print("✓ Results exported to spectral_v2_results.json")